In [ ]:
%load_ext tensorboard

from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import datetime
import pathlib
import os
import gc
from core import Model
import numpy as np
import math
from itertools import permutations

os.environ['CUDA_​DEVICE_​ORDER'] = 'PCI_BUS_ID'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
clock_start = datetime.now().strftime("%Y%m%d-%H%M%S")
#clock_start = "20240722-085506"
workdir = str(pathlib.Path().resolve())


n = 4
cnot_variations = np.empty((math.factorial(n), n), dtype=np.uint8, order='F')
cnot_variations[0, 0] = 0

rows_to_copy = 1
for i in range(1, n):
    cnot_variations[:rows_to_copy, i] = i
    for j in range(1, i + 1):
        start_row = rows_to_copy * j
        end_row = rows_to_copy * (j + 1)
        splitter = i - j
        cnot_variations[start_row: end_row, splitter] = i
        cnot_variations[start_row: end_row, :splitter] = cnot_variations[:rows_to_copy, :splitter]
        cnot_variations[start_row: end_row, splitter + 1:i + 1] = cnot_variations[:rows_to_copy, splitter:i]

    rows_to_copy *= i + 1

ent_variations = list(permutations(np.arange(4), 2))

In [ ]:

def frontload_data():
    from tensorflow import keras
    mnist_dataset = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()
    mnist_dataset = keras.datasets.mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

def pool_job(input):
    this_model = Model(*input)
    this_model.set_data()
    #this_model.prep()
    this_model.prep_ent_cnot_variant(input[-2],input[-1])
    this_model.pre_filter()
    this_model.fit()
    this_model.save_filtered()

#to prevent race condition
frontload_data()

for cnot_var in cnot_variations:
    with ProcessPoolExecutor(8) as executor:
        runner = {
                executor.submit(pool_job, (0,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations
            }
        for future in as_completed(runner):
            runner.pop(future)


In [ ]:

#pool_job((0,2,clock_start,workdir,cnot_variations[0],ent_variations[0]))

In [ ]:

def pool_job2(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    #restored_model.prep()
    restored_model.prep_ent_cnot_variant(detail[-2],detail[-1])
    restored_model.load_history()
    restored_model.visuals.circuit()
    restored_model.visuals.weights()
    restored_model.visuals.relative_weights()
    restored_model.visuals.bias()
    restored_model.visuals.relative_bias()
    restored_model.visuals.data_samples()
    restored_model.visuals.relative_samples()
    restored_model.visuals.filtering()


for cnot_var in cnot_variations:
    with ProcessPoolExecutor(8) as executor:
        runner = {
                executor.submit(pool_job2, (0,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations
            }
        for future in as_completed(runner):
            runner.pop(future) 

In [ ]:

def UMAP_job(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    restored_model.prep()
    restored_model.load_history()
    restored_model.visuals.manifold_umap()

for cnot_var in cnot_variations:
    {UMAP_job((0,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations}